<a href="https://colab.research.google.com/github/Pawanme9034/Sentiment_Analysis_Predicting_sentiment_of_COVID-19_tweets/blob/main/Sentiment_Analysis_Predicting_sentiment_of_COVID_19_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Sentiment Analysis : Predicting sentiment of COVID-19 tweets



##### **Project Type**    - EDA/Classification
##### **Contribution**    - Individual
##### **Name -**          - Pawan Kumar singh


# **Project Summary -**


### This challenge asks you to build a classification model to predict the sentiment of COVID-19 tweets.The tweets have been pulled from Twitter and manual tagging has been done then.

### The names and usernames have been given codes to avoid any privacy concerns.

### You are given the following information:
1. Location
2. Tweet At
3. Original Tweet
4. Label

# **GitHub Link -**

https://github.com/Pawanme9034/Sentiment_Analysis_Predicting_sentiment_of_COVID-19_tweets/blob/main/Sentiment_Analysis_Predicting_sentiment_of_COVID_19_tweets.ipynb

# **Problem Statement**


To build a classification model to predict the sentiment of COVID-19 tweets

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
!pip install transformers -q

In [ ]:
!pip install contractions

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import contractions
import re
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')


### Dataset Loading

In [ ]:
# Load Dataset
import requests
from io import StringIO
# uploading data through Github directly
url = "https://raw.githubusercontent.com/Pawanme9034/Sentiment_Analysis_Predicting_sentiment_of_COVID-19_tweets/main/Coronavirus%20Tweets.csv"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)

df=pd.read_csv(data)

### Dataset First View

In [ ]:
# Dataset First Look
df.head(4)

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns codf.
df.shape

### Dataset Information

In [ ]:
# Dataset Info\
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count.
df.duplicated().sum()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()

In [ ]:
# Visualizing the missing values
null_percentage = df.isnull().mean() * 100

# Display the percentage of null values
print(null_percentage)

### What did you know about your dataset?

1. Original Dataset contains 6 columns and 41157 rows.
2. Location column contains approx 21% of Null values.
3. In order to analyse the data we required only two columns "OriginalTweet" & "Sentiment".
4. The columns such as “UserName” and “ScreenName” does not give any meaningful insights for our analysis.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe()

### Variables Description

Answer Here

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
st_df=df[['OriginalTweet','Sentiment']]

In [ ]:
# We will convert this 5-class classification problem into a 3-class classification problem by merging Extremely Positive tweets with positive tweets and Extremely Negative tweets with negative tweets.
def Sentiment_Grouping(sentiment):
  if sentiment == "Extremely Positive":
    return "Positive"
  elif sentiment == "Extremely Negative":
    return "Negative"
  else:
    return sentiment

In [ ]:
#Apply the Sentiment_Grouping function on the Sentiment column
st_df["Sentiment"] = st_df["Sentiment"].apply(Sentiment_Grouping)

In [ ]:
df['Sentiment'].unique()

In [ ]:
st_df['Sentiment'].unique()

In [ ]:
st_df.head(2)

In [ ]:
#initializes a tokenizer using the BERT base cased model. The BERT tokenizer is used to convert text into tokens that can be processed by the BERT model.
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
tweet_len=[]
for text in st_df.OriginalTweet:
    tokens=tokenizer.encode(text,max_length=512)
    tweet_len.append(len(tokens))
print(max(tweet_len))

canvarting tweetat columns into datetime farmete

In [ ]:
# canvarting tweetat columns into datetime farmete
df['TweetAt']=pd.to_datetime(df['TweetAt'])

In [ ]:
#Obtaining the time period of the dataset
print("Our dataset contains tweets from {}th {} {} to {}th {} {}".format(df["TweetAt"].min().day, df["TweetAt"].min().month_name(), df["TweetAt"].min().year,
         df["TweetAt"].max().day, df["TweetAt"].max().month_name(), df["TweetAt"].max().year))

In [ ]:
#Creating new column for day and month from date colum "TweetAt"
df['day'] = pd.DatetimeIndex(df['TweetAt']).day
df['month'] = pd.DatetimeIndex(df['TweetAt']).month
#Creating a column containing name of the weekday
df["week day"]=pd.DatetimeIndex(df['TweetAt']).day_name()

In [ ]:
#Checking the top 5 row of our dataset
df.head()

Number of tweets in each location

In [ ]:
#Number of tweets in each location
Location_df = df['Location'].value_counts().rename_axis('Location').reset_index(name='Count')
Location_df

In [ ]:
# Assign tweets_per_loc which contains location and number of tweets
tweets_per_loc = df['Location'].value_counts().loc[lambda x: x > 100].sort_values(ascending=False)


In [ ]:
tweets_per_loc

In [ ]:
import seaborn as sns

# Assuming tweets_per_loc is a pandas Series
tweets_per_loc = df['Location'].value_counts().loc[lambda x: x > 100].sort_values(ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x=tweets_per_loc.index, y=tweets_per_loc, palette='viridis')
plt.title('Number of Tweets per Location')
plt.xlabel('Location')
plt.ylabel('Number of Tweets')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.show()


### What all manipulations have you done and insights you found?

1. Extreating 'OriginalTweet','Sentiment' columns and creating st_df datafram
2. initializes a tokenizer using the BERT base cased model. The BERT tokenizer is used to convert text into tokens that can be processed by the BERT model.

3. canvarting tweetAt columns into datetime farmete
4. assing tweets_per_loc which contens loction and number of tweets

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
plot = sns.countplot(x='Sentiment', data=df).set_xticklabels(labels=['Neutral', 'Positive', 'Extremely Negative', 'Negative','Extremely Positive'],rotation=20)

##### 1. Why did you pick the specific chart?

The count plot was chosen for its suitability in visualizing the distribution of categorical sentiment data. It provides a clear and easy-to-interpret representation of the frequency of each sentiment category, allowing for effective comparison.

##### 2. What is/are the insight(s) found from the chart?

The bar chart you sent me shows the results of a sentiment analysis of 10,000 text samples. The samples were classified into five categories: extremely negative, negative, neutral, positive, and extremely positive.

The chart shows that the most common sentiment was neutral, with 4,282 samples (43%). Positive sentiment was the second most common, with 3,120 samples (31%). Extremely negative sentiment was the least common, with 322 samples (3%).

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

In [ ]:
#Plotting the total number of tweets for each subcategory in the target variable after the transformation
plt.rcParams["figure.figsize"]=10,10
sns.countplot(x="Sentiment", data=st_df)

#### Chart - 2

In [ ]:
tweets_per_loc.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Location')
plt.ylabel('Number of Tweets')
plt.title('Number of Tweets per Location')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Python program to generate WordCloud
  ## Based on https://www.geeksforgeeks.org/generating-word-cloud-python/
from wordcloud import WordCloud, STOPWORDS
comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in df.OriginalTweet:

    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code\
f,ax=plt.subplots(figsize=(6,4))
sns.distplot(tweet_len)
plt.xlabel('Number of words')

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
#Plotting the total number of tweets for each subcategory in the target variable after the transformation
plt.rcParams["figure.figsize"]=10,10
sns.countplot(x="Sentiment", data=st_df)

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

Let's create a word cloud showing the different variables in the given dataset.

In [ ]:
# Chart - 6 visualization code
text1 = " ".join(title for title in df.columns)
# Creating word_cloud with text as argument in .generate() method
word_cloud1 = WordCloud(collocations = False, background_color = 'white').generate(text1)
# Display the generated Word Cloud
plt.imshow(word_cloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
def extract_hash_tags(s):
  return [part[1:] for part in s.split() if part.startswith('#')]
df['hashtags']=df['OriginalTweet'].apply(extract_hash_tags)

In [ ]:
from collections import Counter
# Assuming df['hashtags'] is a column containing lists of hashtags in your DataFrame
dh = Counter(df['hashtags'].sum())

# Creating a DataFrame
hashtags = pd.DataFrame({'hashtag': list(dh.keys()), 'Count': list(dh.values())})

# Sorting the DataFrame based on the count
hashtags = hashtags.sort_values(by='Count', ascending=False).reset_index(drop=True)

# Display the sorted DataFrame
print(hashtags)


In [ ]:
import plotly.graph_objects as go
#Creating a pie plot to show the top 10 hashtag and their relative percentage share among themselves
data = {
   "values": hashtags['Count'][:10],
   "labels": hashtags["hashtag"][:10],
   "domain": {"column": 0},
   "name": "hashtags",
   "hoverinfo":"label+percent+name",
   "hole": .4,
   "type": "pie"
}
#Providing title for the plot
layout = go.Layout(title="<b>Relative Percentage of top 10 Hashtag</b>", legend=dict(x=0.9, y=0.8, orientation="v"))
#Ploting
data = [data]
fig = go.Figure(data = data, layout = layout)
fig.update_layout(title_x=0.5)
fig.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
sns.distplot(st_df['OriginalTweet'].str.len())

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code
def show_dist(df, col):

  '''Function is to show distribution of any particular column for each of the different sentiment'''

  bins = np.arange(df[col].min(), df[col].max() + 1)
  g = sns.FacetGrid(df, col='Sentiment', height=5, hue='Sentiment', palette="bright")
  g = g.map(sns.histplot, col, kde=False, bins=bins)

In [ ]:
#Creating a new column named "Number Of Words" to store the number of words in each tweet
df["Number Of Words"] = df["OriginalTweet"].str.split().apply(len)

show_dist(df , 'Number Of Words')

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code
#Plotting total number of tweets for each sentiment over the months
plt.rcParams["figure.figsize"]=(10,9)
ax = sns.countplot(x="month", hue="Sentiment", data=df)
plt.ylabel("Number of tweets")
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code
#Creating a subset of our dataframe that will only contains tweets from the month of march
day_analysis = df[df["month"] == 3].copy()
#Grouping our dataset on the basis of Day and sentiment to get the number of tweets tweeted on each day in the month of march belonging to each sentiment
daySentiment_groupby = day_analysis.groupby(["day" , "Sentiment"]).agg({ "OriginalTweet" : "count"}).reset_index()
sns.lineplot(x = "day" , y ="OriginalTweet" , hue="Sentiment" ,  data = daySentiment_groupby , palette = 'bright' )
plt.title("Number of tweets in the month of March")
plt.ylabel("Number of tweets")
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

Let's check out the day when people tweet the most.

In [ ]:
# Chart - 12 visualization code
#Checking the day on which more tweets are being created
ax = sns.countplot(x="week day", hue="Sentiment", data=df)

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code
#Creating a new column named "Number Of Mentions" to store the number of mentions in each tweet
df["Number Of Mentions"] = df["OriginalTweet"].apply(lambda tweet : len(re.findall(r'@[^\s]+', tweet)))

show_dist(df, "Number Of Mentions")

In [ ]:
#Creating a new columns named "Number Of Hashtags" to store the number of hastags in each tweet
df["Number Of Hashtags"] = df["OriginalTweet"].apply(lambda tweet : len(re.findall(r'#[^\s]+', tweet)))

show_dist(df, "Number Of Hashtags")

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap


Let's create a word cloud to visualize the hashtags

In [ ]:
# Correlation Heatmap visualization code
#Creating a dictionary containing hash tag and it's count
data = dict(zip(hashtags['hashtag'].tolist(), hashtags['Count'].tolist()))
#Generate word cloud of hash tags based on their frequency
wc = WordCloud(width=800, height=400, max_words=200,background_color = 'black').generate_from_frequencies(data)
#Providing figure size
plt.figure(figsize=(10, 10))
#plotting
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code
df.head()

In [ ]:
#Apply the Sentiment_Grouping function on the Sentiment column
df["Sentiment"] = df["Sentiment"].apply(Sentiment_Grouping)

In [ ]:
#Creating subsets of the dataframe based on sentiment category Positive, Negative and Neutral
Positive_tweet = df[df["Sentiment"] == "Positive"]
Negative_tweet = df[df["Sentiment"] == "Negative"]
Neutral_tweet = df[df["Sentiment"] == "Neutral"]

In [ ]:
def hashtag_freq_graph(df):
  '''This function gives the frequency bar graph of the hashtags'''
  #Extracting hashtags from tweets
  df['hashtag'] = df['OriginalTweet'].apply(lambda twt : re.findall(r"#(\w+)", twt))
  #Taking value count on hashtags
  hashtags=df.OriginalTweet.str.extractall(r'(\#\w+)')[0].value_counts().reset_index()
  #renaming the columns
  hashtags.rename(columns={'index':"hashtag",0:'freq'},inplace=True)

  #Keeping only 15  most frequently used hashtags
  hashtags=hashtags[:15]
  #Seting figuresize
  sns.set(rc = {'figure.figsize':(15,8)})
  #Ploting frequency against hashtag
  sns.barplot(x="hashtag" , y ="freq" , data = hashtags , palette = "bright")
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
#Calling hashtag_freq_graph on positive tweet dataframe
hashtag_freq_graph(Positive_tweet)

In [ ]:
#Calling hashtag_freq_graph on negative tweet dataframe
hashtag_freq_graph(Negative_tweet)

In [ ]:
#Calling hashtag_freq_graph on neutral tweet dataframe
hashtag_freq_graph(Neutral_tweet)

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation
st_df.isnull().sum()

#### What all missing value imputation techniques have you used and why did you use those techniques?

no missing values in working dataset

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

no outliers in working dataset

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
# creating a function to repalce the 'Sentiment' value
def sentiment_categorie(category):
    if category=='Extremely Negative' or category=='Negative':
        return 0
    elif category=='Neutral':
        return 1
    else:
        return 2

In [ ]:
df['Sentiment']=df['Sentiment'].apply(sentiment_categorie)

#### What all categorical encoding techniques have you used & why did you use those techniques?

replacing the values of 'Sentiment' column

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

Contraction is an abbreviated form of a word, like don't stand for do not. This contraction in the text data needs to be expanded for better analysis. It will match the word with the keys, and if it exists it will replace the word with its value.

In [ ]:
# Expand Contraction

def expand_contractions(text):
    return contractions.fix(text)

def expand_all_contractions(df, column_name):
    expanded_texts = []
    for text in df[column_name]:
        expanded_texts.append(expand_contractions(text))
    return expanded_texts

# Apply the function on the st_df dataset
df['OriginalTweet'] = expand_all_contractions(df, 'OriginalTweet')




#### 2. Lower Casing

In [ ]:
# Lower Casing
# Convert the text in 'OriginalTweet' column to lowercase
df['OriginalTweet'] = df['OriginalTweet'].str.lower()


#### 3. Removing Punctuations

In [ ]:
# Remove punctuations from the text in 'OriginalTweet' column
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'[^\w\s]', '', regex=True)


#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

# Remove URLs from the text in 'OriginalTweet' column
st_df['OriginalTweet'] = st_df['OriginalTweet'].str.replace(r'http\S+|www\S+', '', regex=True)


#### 5. Removing Stopwords & Removing White spaces

In [ ]:
from nltk.corpus import stopwords
# Remove stopwords from the text in 'OriginalTweet' column
stopwords_set = set(stopwords.words('english'))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords_set]))

# Remove white spaces from the text in 'OriginalTweet' column
df['OriginalTweet'] = df['OriginalTweet'].str.strip()


In [ ]:
# Remove white spaces from the text in 'OriginalTweet' column
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'http\S+', ' ', regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'<.*?>', ' ', regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'\d+', ' ',  regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'#\w+', ' ', regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'@\w+', ' ', regex=True).str.strip()
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'_', ' ', regex=True).str.strip()


In [ ]:
df['OriginalTweet']

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize

df['TokenizedTweet'] = df['OriginalTweet'].apply(word_tokenize)


In [ ]:
df['TokenizedTweet']

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)
from nltk.stem.snowball import SnowballStemmer

In [ ]:
# create an object of stemming function
stemmer = SnowballStemmer("english")

def stemming(text):
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [ ]:
df['OriginalTweet']= df['OriginalTweet'].apply(stemming)

In [ ]:
df['OriginalTweet']

In [ ]:

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatization(text):
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(text)

# Assuming `st_df` is your pandas DataFrame with a column named 'OriginalTweet'
df['OriginalTweet'] = df['OriginalTweet'].apply(lemmatization)


In [ ]:
df.head()

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

def pos_tagging(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    return pos_tags



In [ ]:
df['TokenizedTweet'] = df['OriginalTweet'].apply(pos_tagging)

In [ ]:
df['OriginalTweet']

#### 10. Text Vectorization

In [ ]:
# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the 'OriginalTweet' column
X_tfidf = vectorizer.fit_transform(df['OriginalTweet']).toarray()
X = vectorizer.fit_transform(df['OriginalTweet']).toarray()
y = df["Sentiment"]

In [ ]:
df

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

#### 2. Feature Selection

##### What all feature selection methods have you used  and why?

Answer Here.

##### Which all features you found important and why?

Answer Here.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data


##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

### 9. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1,stratify=y)

In [ ]:
 X_train.shape, y_train.shape, X_test.shape, y_test.shape

##### What data splitting ratio have you used and why?

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
#Creating an instance of the model LogisticRegression
logreg = LogisticRegression()
#Fitting the model on the train dataset
logreg.fit(X_train, y_train)

#Making prediction on our train and test dataset
logreg_pred_train = logreg.predict(X_train)
logreg_pred_test = logreg.predict(X_test)

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart
#Storing train and test accuracy scores
logreg_train_accuracy = accuracy_score(y_train,logreg_pred_train)
logreg_test_accuracy = accuracy_score(y_test,logreg_pred_test)
#printing train and test accuracy scores
print("Training accuracy Score    : ",logreg.score(X_train,y_train))
print("Test accuracy Score : ",logreg_test_accuracy )
#printing classification report for the performance of train and test set
print("\n","LogisticRegression-Classification Report for train dataset","\n")
print(classification_report(logreg_pred_train,y_train))
print("LogisticRegression-Classification Report for test dataset","\n")
print(classification_report(logreg_pred_test,y_test))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:

# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
# Preprocess the text data using TF-IDF vectorization

param_grid = {

    'solver': [ 'saga','newton-cg','lbfgs', 'liblinear', ],
    'max_iter': [100,10000]
}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(logreg, param_grid, cv=5)

# Fit the GridSearchCV on the preprocessed data
grid_search.fit(X_train_tfidf, y_train)

# Access the best parameters, best score, and best estimator
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_estimator = grid_search.best_estimator_


In [ ]:
print(best_params )
print(best_score )
print(best_estimator )

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
#Creating an instance of the model LinearSVC
SVC = LinearSVC()
#Fitting the model on the train dataset
SVC.fit(X_train, y_train)

#Making prediction on our train and test dataset
SVC_pred_train = SVC.predict(X_train)
SVC_pred_test = SVC.predict(X_test)


In [ ]:
# Visualizing evaluation Metric Score chart
#Storing train and test accuracy scores
SVC_train_accuracy = accuracy_score(y_train,SVC_pred_train)
SVC_test_accuracy = accuracy_score(y_test,SVC_pred_test)

#printing train and test accuracy scores
print("Training accuracy Score    : ",SVC.score(X_train,y_train))
print("Test accuracy Score : ",SVC_test_accuracy )

#printing classification report for the performance of train and test set
print("\n","LinearSVC-Classification Report for train dataset","\n")
print(classification_report(SVC_pred_train,y_train))
print("LinearSVC-Classification Report for test dataset","\n")
print(classification_report(SVC_pred_test,y_test))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': [0.1, 1, 10]
}
# Fit the Algorithm
grid_search = GridSearchCV(estimator=SVC, param_grid=param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

# Predict on the model
best_params_svm = grid_search.best_params_
best_score_svm = grid_search.best_score_


In [ ]:
print(best_params_svm)
print(best_score_svm)

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# ML Model - 3 Implementation
#Creating an instance of the model MultinomialNB
naive_bayes= MultinomialNB()
#Fitting the model on the train dataset
naive_bayes.fit(X_train,y_train)

#Making prediction on our train and test dataset
nb_pred_train = naive_bayes.predict(X_train)
nb_pred_test = naive_bayes.predict(X_test)


#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#Storing train and test accuracy scores
nb_train_accuracy=accuracy_score(y_train,nb_pred_train)
nb_test_accuracy=accuracy_score(y_test,nb_pred_test)

#printing train and test accuracy scores
print("Training accuracy Score    : ", model.score(X_train,y_train))
print("Test accuracy Score : ", nb_test_accuracy )

#printing classification report for the performance of train and test set
print("\n","Naive Bayes-Classification Report for train dataset","\n")
print(classification_report(y_train,nb_pred_train))
print("Naive Bayes-Classification Report for test dataset","\n")
print(classification_report(y_test,nb_pred_test))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'fit_prior': [True, False],
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(naive_bayes, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_vectorized, y_train)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set using the best hyperparameters
best_naive_bayes = grid_search.best_estimator_
y_pred = best_naive_bayes.predict(X_test_vectorized)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
print(best_params_rf)
print(best_score_rf)

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.
Tweet = ['hospitals are good', 'boy is sad',
         'he fell sick', 'he is not satisfied',
        'he is angry with his behaviour', 'taste of food bad', 'snacks are good']
result = model2.predict(Tweet)


def reverse_sentiment_categories(category):
    if category == 0:
        return 'Negative'
    elif category == 1:
        return 'Neutral'
    else:
        return 'Positive'

result = [reverse_sentiment_categories(category) for category in result]

# Print the results
print(result)

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***